In [2]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE
using FUSE.IMAS

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [2]:
core_profiles = IMAS.core_profiles();
resize!(core_profiles.profiles_1d,1)
profiles_1d = core_profiles.profiles_1d[1]

profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=21)
profiles_1d.electrons.density = (1.0 .- profiles_1d.grid.rho_tor_norm.^2);
profiles_1d.electrons.temperature = (x;_...) -> (1.0 .- x.^2);

#println(profiles_1d.electrons.pressure)
profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=5)
#println(profiles_1d.electrons.temperature)

"""
    goto(ids::IDS, location::String)

Reach location in a given IDS
"""
function goto(ids::IDS, location::String; f2::Function=f2i)
    # find common ancestor
    cs, s1, s2 = IMAS.common_base_string(f2(ids), location)
    cs0 = replace(cs, r"\.$" => "")
    # go upstream until common acestor
    h = ids
    while f2(h) != cs0
        println("= ",f2(h))
        println("- ",cs0)
        h = h._parent.value
        if h === missing
            error("Could not reach `$(location)` from `$(f2(ids))`")
        end
    end
    # then dive into the location branch
    for k in i2p(s2)
        h = getfield(h, Symbol(k))
    end
    return h
end

#goto(profiles_1d.electrons,"core_profiles.profiles_1d[].grid.rho_tor_norm")
#println(f2u(profiles_1d.electrons))

goto

In [2]:
FUSE.IMAS.equilibrium(1)

LoadError: MethodError: no method matching IMAS.equilibrium(::Int64)
[0mClosest candidates are:
[0m  IMAS.equilibrium(::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at /Users/meneghini/.julia/dev/IMAS/src/dd.jl:10109
[0m  IMAS.equilibrium(; _uninitialized) at /Users/meneghini/.julia/dev/IMAS/src/dd.jl:10117
[0m  IMAS.equilibrium([91m::IDSvector{T} where T<:IMAS.equilibrium__time_slice[39m, [91m::Union{Missing, Function, Vector{T} where T<:Real}[39m, [91m::IMAS.equilibrium__ids_properties[39m, [91m::IDSvector{T} where T<:IMAS.equilibrium__grids_ggd[39m, [91m::IMAS.equilibrium__vacuum_toroidal_field[39m, [91m::IMAS.equilibrium__code[39m, [91m::WeakRef[39m) at /Users/meneghini/.julia/dev/IMAS/src/dd.jl:10109

In [3]:
R0 = 1.8
δ = 0.5
ϵ = 0.32
κ = 1.7
B0 = 2.0
qstar = 1.57

println(FUSE.init(IMAS.equilibrium(), 0.0; B0, R0, ϵ, δ, κ, qstar))

time_slice
｜[1]
｜｜profiles_1d
｜｜｜elongation ➡ 1-element Vector{Float64}
｜｜｜psi ➡ 1-element Vector{Float64}
｜｜｜triangularity_upper ➡ 1-element Vector{Float64}
｜｜｜r_outboard ➡ 1-element Vector{Float64}
｜｜｜triangularity_lower ➡ 1-element Vector{Float64}
｜｜｜r_inboard ➡ 1-element Vector{Float64}
｜｜｜q ➡ 1-element Vector{Float64}
｜｜global_quantities
｜｜｜energy_mhd ➡ IMAS.var"#10#12"
time ➡ 1-element Vector{Float64}
vacuum_toroidal_field
｜b0 ➡ 1-element Vector{Float64}
｜r0 ➡ Float64



In [11]:
#= ============ =#
#  AbstractActor #
#= ============ =#

abstract type AbstractActor end

function step(actor::AbstractActor, ids::IDS, time_index::Integer)
    error("Function step() not defined for actor of type $(typeof(actor))")
end

function finalize(actor::AbstractActor, ids::IDS, time_index::Integer)
    error("Function finalize() not defined for actor of type $(typeof(actor))")
end

#= =========== =#
#  Equilibrium  #
#= =========== =#

using Equilibrium

abstract type EquilibriumActor <: AbstractActor end

struct SolovevEquilibriumActor <: EquilibriumActor
    S :: SolovevEquilibrium
end

function SolovevEquilibriumActor(equilibrium::IMAS.equilibrium, time_index::Integer)
    eqt=equilibrium.time_slice[time_index]
    a = eqt.profiles_1d.r_outboard[end] - eqt.profiles_1d.r_inboard[end]
    R0 = eqt.profiles_1d.r_outboard[end] + eqt.profiles_1d.r_inboard[end]
    κ = eqt.profiles_1d.elongation[end]
    δ = (eqt.profiles_1d.triangularity_lower[end] + eqt.profiles_1d.triangularity_upper[end])/2.0
    ϵ = a/R0
    B0 = abs(equilibrium.vacuum_toroidal_field.b0[time_index])
    B0_dir = Int(sign(B0))
    R0 = equilibrium.vacuum_toroidal_field.r0
    qstar = eqt.profiles_1d.q[end]
    Ip_dir = Int(sign(qstar)*B0_dir)
    alpha = -0.155
    S = solovev(B0, R0, ϵ, δ, κ, alpha, qstar, B0_dir=B0_dir, Ip_dir=Ip_dir)
    return SolovevEquilibriumActor(S)
end

equilibrium_actor = SolovevEquilibriumActor(equilibrium, 1)

LoadError: UndefVarError: equilibrium not defined

In [29]:
a=Array{Float64,0}[]
a[]=5

LoadError: BoundsError: attempt to access 0-element Vector{Array{Float64, 0}} at index []